In [1]:
!pip install nltk

In [16]:
!pip install itables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00


In [2]:
!pip install spacy

In [3]:
#required libraries
from google.colab import files
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
#uploading the Aircraft_replaced_abbreviation to extract required terms

def read_dataset(csv_file):
  Aircraft=pd.read(csv_file)
  return Aircraft
dataset=files.upload()

Saving Annotated problems Aircraft Data1 - Blad1.csv to Annotated problems Aircraft Data1 - Blad1.csv


In [40]:
import csv
import json
import spacy
import random
def process_csv(input_file):
    spacy_training_data = []
    with open(input_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            text = row['PROBLEM']
            entities = []
            for entity_type in ['LOCATION', 'PART', 'TGGEDPROBLEM']:
                entity = row[entity_type]
                if entity != 'None':
                    start = text.find(entity)
                    end = start + len(entity)
                    entities.append((start, end, entity_type))
            spacy_training_data.append((text, {"entities": entities}))
    return spacy_training_data

nlp = spacy.blank("en")

ner = nlp.add_pipe("ner")
ner.add_label("LOCATION")
ner.add_label("PART")
ner.add_label("TGGEDPROBLEM")

input_file = 'Annotated problems Aircraft Data1 - Blad1.csv'


spacy_training_data = process_csv(input_file)
training_data = spacy_training_data[:400]



In [41]:
from spacy.training import Example
#Example class is used to represent training examples for training(NER) model. Example  consists of a text document and the annotations (entities).
from spacy.util import minibatch
examples = []
for text, annot in training_data:
    doc = nlp.make_doc(text)
    entities = annot.get("entities")
    spans = []
    for start, end, label in entities:
        span = doc.char_span(start, end, label=label)
        if span is None:
            print(f"Failed to create span for '{text}' with label '{label}', skipping this annotation.")
        else:
            spans.append(span)
    if spans:
        try:
            examples.append(Example.from_dict(doc, {"entities": spans}))
        except ValueError as e:
            print(f"Ignoring annotation due to error: {e}")


# Train the SpaCy NER model
nlp.begin_training()
for epoch in range(10):
    random.shuffle(examples)
    #minibatch is a utility function for batching training data during the training process. It allows you to efficiently process training examples in batches
    for batch in minibatch(examples, size=8):
        nlp.update(batch)

# Save the trained model to a directory
output_dir = "ner_model"
nlp.to_disk(output_dir)
print("Trained NER model saved to:", output_dir)


Failed to create span for 'L/H ENGINE #4 AFT BAFFL ECRACKED IN MULTIPLE PLACES.' with label 'TGGEDPROBLEM', skipping this annotation.
Ignoring annotation due to error: [E1010] Unable to set entity information for token 7 which is included in more than one span in entities, blocked, missing or outside.
Failed to create span for 'STUDENT COULDN'T START A/C.' with label 'TGGEDPROBLEM', skipping this annotation.
Ignoring annotation due to error: [E1010] Unable to set entity information for token 2 which is included in more than one span in entities, blocked, missing or outside.
Failed to create span for 'TOP FRONT R/H BAFFLE, BAFFLE SEAL RIVET PULLED THROUGH.' with label 'TGGEDPROBLEM', skipping this annotation.
Ignoring annotation due to error: [E1010] Unable to set entity information for token 0 which is included in more than one span in entities, blocked, missing or outside.
Ignoring annotation due to error: [E1010] Unable to set entity information for token 2 which is included in more 

In [42]:
# Load the trained model from the directory
loaded_nlp = spacy.load(output_dir)

# Process the next 100 rows from your data and convert them into SpaCy Example objects
training_data = spacy_training_data[450:500]
next_100_examples = []
for text, annot in training_data:
    doc = loaded_nlp.make_doc(text)
    entities = annot.get("entities")
    spans = []
    for start, end, label in entities:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            spans.append(span)
    if spans:
        next_100_examples.append(Example.from_dict(doc, {"entities": spans}))

# Evaluate the performance of the model
scores = loaded_nlp.evaluate(next_100_examples)
print("Evaluation scores:", scores)

Evaluation scores: {'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.9166666666666666, 'ents_r': 0.9307692307692308, 'ents_f': 0.9236641221374045, 'ents_per_type': {'LOCATION': {'p': 0.9047619047619048, 'r': 0.8837209302325582, 'f': 0.8941176470588236}, 'PART': {'p': 0.8863636363636364, 'r': 0.9512195121951219, 'f': 0.9176470588235294}, 'TGGEDPROBLEM': {'p': 0.9565217391304348, 'r': 0.9565217391304348, 'f': 0.9565217391304348}}, 'speed': 16182.071277499706}


In [36]:
import itables
from itables import show

In [43]:
# Load the trained model from the directory
loaded_nlp = spacy.load(output_dir)

# Process the remaining rows and tokenize them using the loaded model
remaining_data = spacy_training_data[500:]
predicted_entities = []
for idx, (text, _) in enumerate(remaining_data, start=501):
    doc = loaded_nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    predicted_entities.append((idx, text, entities))

# Save the predicted entities to a file
predicted_file = 'predicted_entities.csv'
with open(predicted_file, 'w') as f:
    f.write("Row,Entities\n")
    for idx, text, entities in predicted_entities:
        entity_str = ";".join([f"{entity[0]}:{entity[1]}" for entity in entities])
        f.write(f"{idx},{text},{entity_str}\n")

print("Predicted entities saved to:", predicted_file)

for idx, text, entities in predicted_entities:
    print(f"Row {idx}:")
    print("Text:", text)
    for entity, label in entities:
        print(f"Entity: {entity}, Label: {label}")


Streaming output truncated to the last 5000 lines.
Row 5014:
Text: BLACK FWD SEAL HAS WORN SPOTS.
Entity: FWD SEAL, Label: PART
Entity: WORN, Label: TGGEDPROBLEM
Row 5015:
Text: L/H ENG CYL #4 INTAKE GASKET IS LEAKING.
Entity: L/H, Label: LOCATION
Entity: CYL #4, Label: LOCATION
Entity: INTAKE GASKET, Label: PART
Entity: LEAKING, Label: TGGEDPROBLEM
Row 5016:
Text: 1 BAFFLE PLUG IS DAMAGED.
Entity: BAFFLE PLUG, Label: PART
Entity: DAMAGED, Label: TGGEDPROBLEM
Row 5017:
Text: RPM 2700 FOR 2 SEC.
Entity: 2 SEC, Label: TGGEDPROBLEM
Row 5018:
Text: PROP OVERSPEED 2750 FOR 5 SECS.
Entity: OVERSPEED, Label: TGGEDPROBLEM
Row 5019:
Text: UNUSUAL ATTITUDE POWER WAS ADDED & ENGINE REACHED APPROX 280
Entity: ENGINE, Label: LOCATION
Row 5020:
Text: UNUSUAL ATTITUDE POWER WAS ADDED & ENGINE REACHED APPROX 280
Entity: ENGINE, Label: LOCATION
Row 5021:
Text: UNUSUAL ATTITUDE POWER WAS ADDED & ENGINE REACHED APPROX 280
Entity: ENGINE, Label: LOCATION
Row 5022:
Text: BAFFLE SCREW MISSING ON #3 CYL.
Ent

In [44]:
# Initialize a list to store parts
parts = []

# Iterate over predicted entities and extract parts
for _, _, entities in predicted_entities:
    for entity, label in entities:
        if label == 'PART':
            parts.append(entity)

# Print the extracted parts
print("Extracted parts:")
for part in parts:
    print(part)

Extracted parts:
BAFFLE
ROCKER COVER GASKETS
INTAKE GASKET
INTAKE GASKET
INTAKE
ROCKER COVER GASKET
ROCKER COVER GASKET
INTAKE GASKET
BAFFLE PLUGS
ROCKER COVER GASKETS
INTAKE GASKETS
BAFFLE SPARK PLUG
INTAKE GASKETS
ROCKER COVER GASKET
INTAKES
ROCKER COVER
INTAKE GASKET
ROCKER COVER GASKET
INTAKE PIPE GASKET
AFT BAFFLE
INCRE
OIL RETURN LINE
INTAKE GASKET
COWL SHOCK MOUNTS
ROCKER COVER GASKETS
ROCKER BOX COVER SCREWS
ROCKER BOX COVER SCREWS
INDUCTION TUBE HOSE CLAMPS
BAFFLE
INTAKE
ROCKER COVER GASKET
INTAKE GASKET
ROCKER COVER
INTAKE GASKET
INTAKE GASKET
BAFFLE PLUG
INTAKE
INTAKES
ROCKER COVERS
INDUCTION TUBE GASKET
ROCKER BOX COVER SCREWS
BAFFLE
INTAKES
ROCKER COVERS
ROCKER COVERS
INTAKE
ROCKER COVER
INTAKE
INTAKE TUBE GASKET
OIL RETURN LINE
OIL FILL TUBE
BAFFLE PLUG
INTAKE GASKET
ROCKER COVER GASKETS
INTAKE
INTAKE
ROCKER COVERS
INTAKES
ROCKER COVERS
BAFFLE
ROCKER COVER GASKETS
ROCKER DRAIN
BAFFLE
INTAKE GASKETS
INTAKE GASKET
ROCKER COVER GASKET
INTAKE GASKET
ROCKER COVER GASKET
BAFFLE

In [46]:
# Convert the list of parts to a set to remove duplicates
distinct_parts_set = set(parts)

# Convert the set back to a list if needed
distinct_parts = list(distinct_parts_set)

# Print the distinct parts
print("Distinct parts:")
for part in distinct_parts:
    print(part)


Distinct parts:
LWR AFT COWLING
INTERBAFFLE
BAFFLE ATTACH BRACKET
MAG ON
INLET SCREEN
BAFFLE ABOVE
MAG DROPPED
INTAKE PIPE GASKET
AFT) SEAL
AIR INLET TUBE
AIR FILTER
ROCKER BOX
AIR INTAKE
INDUCTION TUBE
OUT 1 OR
BAFFLE BRACKET
INNER SEAL
ROCKER BOX COVER IS
BAFFLE REPAIR
BAFFLE TENSION
INLET GASKETS
ROCKER ARM
OIL
2 EX
DEPARTMENT
BAFFLE SPARK PLUG
ROCKER COVER SEAL
DOOR
INTAKES
OIL RETURN
BAFFLE PATCH
OIL RETURN TUBE
AIR INTAKE HOSE NOT HOOKED UP
AFT BAFFLING
I/
VERIFY CONDITION
BAFFLES
INTERMITTENTLY
OIL PLUG
OIL BAFFLE COOLER BRACKET
PLUG
OIL RETURN TUBE HOSE CLAMPS
NO
INTAKE HOSE CLAMPS
MAG DROP OF
OIL FILL TUBE
INTAKE TUBE GASKET
AFT BAFFLE
OUT OF TOP L/H
OIL COOLER
TRIED COLD START
VALVE COVER
BAFFLE BARREL NUT
BAFFLE FASTENER
AIR FILTER,
ROCKER GASKETS
OUT OF
OIL COOLER HOSE
AFT BAFFLES
250 RPM @ HOVER
OIL PRESSURE LINE
BAFFLE SUPPORT BRACKET
VALVE COVER GASKETS
AFT BAFFLE SUPPORT
BUT CARB
UPPER COVER
OIL COOLERS
OIL OBSERVE
INLET TUBE
AFT OF
AFT BAFFLE OF
LWR RUBBER DRAIN
AIR BO

In [47]:
# Initialize a list to store parts
problems = []

# Iterate over predicted entities and extract parts
for _, _, entities in predicted_entities:
    for entity, label in entities:
        if label == 'TGGEDPROBLEM':
            problems.append(entity)

# Print the extracted parts
print("Extracted parts:")
for problem in problems:
    print(problem)

Extracted parts:
CRACKED
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
LEAKS
LEAKING
BROKEN
LEAKING
LEAKING
WORN
LEAKING
CAUSING
TORN
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
CRACKED
CRACKED
A/C NORMALLY
LEAKING
LEAKING
TORN
LEAKING
LOOSE
LOOSE
LOOSE
CRACKED
LEAKING
LEAKING
LEAKS
LOOSE
LEAKING
MISSING
ROUGH
MISSING
ROUGH
MISSING
ROUGH
MISSING
ROUGH
LEAKS
MISSING
LEAKING
LEAKING
LEAKING
LEAKING
LOOSE
NO COMPRESSION
NO COMPRESSION
NO COMPRESSION
BROKEN
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
CHAFING
LOOSE
BROKEN
LEAKING
LEAKING
LEAKS
LEAKS
LOOSE
LEAKING
LEAKING
CRACKED
LEAKING
LEAKS
LOOSE
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
CRACKED
LOOSE
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
LOOSE
LOOSE
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
LEAKING
LOOSE
LEAKING
LEAKING
LEAKING
LEAKING
CRACK
LEAKING
LEAKING
LEAKING
LOOSE
LEAKING
CRACK
LEAKING
LEAKING
LEAKING
LOOSE
BROKEN
LEAKING
LOOSE
LEAKING
LEAKING
LEAKING
LEAKING
BROKEN
LEAKS
LEAKING
CRACKED
LEAKING
LEAKING
MISSING
LEA

In [48]:
# Convert the list of parts to a set to remove duplicates
distinct_problems_set = set(problems)

# Convert the set back to a list if needed
distinct_problems = list(distinct_problems_set)

# Print the distinct parts
print("Distinct problems:")
for problem in distinct_problems:
    print(problem)

Distinct problems:
WILL NOT MAKE
DIRTY
54/80)
WEAK
COULD NOT GET V
COWLING
CANNED
HIGH
DUCTING
LITTLE
VIBRATIONS
REMVOED AROUND ENG GROUNDING
OIL ON
HITTING
OVER REVED
WAITING
COMING
50/80)
OVER
DETERIORATED
A/C BEGAN TO LEAK
COOLING
WOULD CAUSE SPUTTER
LOSS
EDGES
WOULD TURN TWICE
FELL OFF
DIRTY BETWEEN FINS
WOULD NOT INCREASE PAST
NO MORE THAN 10
DING
MADE
HOLDING SHORT.
2730
LIQUID
LIST
TEARING OFF
2740
A/C NEEDS AFTER STORAGE
BROKE
RUN AT IDLE
OIL COOLER
ELONGATED
WOULD NOT STAY
LOW COMPRESSION
A/C WOULD
MAGNETO
HANGING FRONT
CYLINDERS
GOING
CHAFED
LACING
NOTED
ABOUT 2 SECONDS
BAD COMPRESSION
ABOUT 2
2 SEC
FUEL
WRONG
ZERO COMPRESSION
COMPRESSION 40/80
CHAFING
A/C WILL NOT START
WILL NOT START
DONE
MELTED
A
FLUCTUATING
D
WOULD SPUTTER WHEN
COWL DZUS
20/80
CHIP
INDUCTION
INCREASE IN THROTTLE
COMPRESSIONS
WOULD FLUCTUATE
CROSSTIE BROKEN
REQUIRED
HOLE
CRACK
NO TORQUE SEAL
CLIMB
1300
WOULD NOT LATCH
CORNER
DAMAGE
HIGH PITCHED WHINE
HIGH PITCHED SQUEALING
A 1 FOOT HOVER @ LIMITING
A/C WOU

In [28]:
loaded_nlp = spacy.load(output_dir)

# Process the remaining rows from your data and tokenize them using the loaded model
remaining_training_data = spacy_training_data[500:]
predicted_entities = []
for text, _ in remaining_training_data:
    doc = loaded_nlp(text)
    predicted_entities.append([(ent.text, ent.label_) for ent in doc.ents])

# Print the predicted entities and their labels
for idx, entities in enumerate(predicted_entities, start=501):
    print(f"Entities in row {idx}: {entities}")

KeyboardInterrupt: 